<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

#  Multi-Query-Retriever

This method can automatically use a ChatModel to make slight variations of your initial query to help attempt overcome any issues with cosine similarity distances, this allows you to phrase things more like a general query question rather than a pure document similarity look-up.

In [8]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

### Loader

In [17]:
loader = WikipediaLoader(query='MKUltra')
documents = loader.load()

In [19]:
len(documents)

9

### Split Documents

In [20]:
# split it into chunks
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
docs = text_splitter.split_documents(documents)

### OpenAI Connection for Embeddings

In [21]:
import os
f = open('C:\\Users\\Marcial\\Desktop\\desktop_openai.txt')
os.environ['OPENAI_API_KEY'] = f.read()
embedding_function = OpenAIEmbeddings()

In [23]:
# docs

### Embed Documents for ChromaDB

In [45]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function,persist_directory='./mk_ultra')
db.persist()

### Use Chat Model to Multi Query

In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
question="When was this declassified?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=db.as_retriever(),llm=llm)

In [40]:
# Set logging for the queries
import logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

In [41]:
unique_docs = retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the date of declassification for this information?', '2. Can you provide the declassification date for this?', '3. At what time was this information officially declassified?']


In [42]:
len(unique_docs)

5

In [43]:
print(unique_docs[0].page_content)

The United States President's Commission on CIA Activities within the United States was ordained by President Gerald Ford in 1975 to investigate the activities of the Central Intelligence Agency and other intelligence agencies within the United States. The Presidential Commission was led by Vice President Nelson Rockefeller, from whom it gained the nickname the Rockefeller Commission.
The commission was created in response to a December 1974 report in The New York Times that the CIA had conducted illegal domestic activities, including experiments on US citizens, during the 1960s. The commission issued a single report in 1975, touching upon certain CIA abuses including mail opening and surveillance of domestic dissident groups. It also publicized Project MKUltra, a CIA mind control research program.
Several weeks later, committees were established in the House and Senate for a similar purpose. White House Personnel, including future Vice President Dick Cheney, edited the results, exclud